In [1]:
import tfx
tfx.version.__version__

'0.13.0'

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'1.13.1'

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration import pipeline
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input

/usr/local/lib/python3.5/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [3]:
%%bash
# download taxi data
mkdir -p taxi/simple
mkdir -p serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O taxi/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O taxi/taxi_utils.py

--2019-06-11 10:07:01--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922668 (1.8M) [text/plain]
Saving to: ‘taxi/simple/data.csv’

     0K .......... .......... .......... .......... ..........  2% 1.97M 1s
    50K .......... .......... .......... .......... ..........  5% 2.12M 1s
   100K .......... .......... .......... .......... ..........  7% 3.03M 1s
   150K .......... .......... .......... .......... .......... 10% 2.84M 1s
   200K .......... .......... .......... .......... .......... 13% 4.09M 1s
   250K .......... .......... .......... .......... .......... 15% 3.57M 1s
   300K .......... .......... .......... .......... .......... 18% 4.33M 1s
   350K .......... ........

In [4]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

In [5]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
example_gen = CsvExampleGen(input_base=examples)

In [6]:
from google.protobuf import json_format
from tfx.proto import example_gen_pb2

input_dict = example_gen.input_dict
output_dict = example_gen.outputs.get_all()
exec_properties = {
    'input' : 
    json_format.MessageToJson(
        example_gen_pb2.Input(splits=[
            example_gen_pb2.Input.Split(
                name='taxi', pattern='data/simple/*'),
        ])),
    'output':
    json_format.MessageToJson(
        example_gen_pb2.Output(
            split_config=example_gen_pb2.SplitConfig(splits=[
                example_gen_pb2.SplitConfig.Split(
                    name='train', hash_buckets=2),
                example_gen_pb2.SplitConfig.Split(
                    name='eval', hash_buckets=1)
            ])))
  }


example_gen.Do(input_dict, output_dict, exec_properties)

AttributeError: module 'tfx.proto.example_gen_pb2' has no attribute 'Input'

In [12]:
from  tfx.proto import example_gen_pb2

# Input train split is 'input_dir/train/*', eval split is 'input_dir/eval/*'.
# Output splits are generated one-to-one mapping from input splits.
input = example_gen_pb2.Input(splits=[
                example_gen_pb2.Input.Split(name='train', pattern='train/*'),
                example_gen_pb2.Input.Split(name='eval', pattern='eval/*')
            ])
examples = csv_input(input_dir)

AttributeError: module 'tfx.proto.example_gen_pb2' has no attribute 'Input'

In [9]:
!cat /usr/local/lib/python3.5/dist-packages/tfx/proto/example_gen_pb2.py

# Generated by the protocol buffer compiler.  DO NOT EDIT!
# source: tfx/proto/example_gen.proto

import sys
_b=sys.version_info[0]<3 and (lambda x:x) or (lambda x:x.encode('latin1'))
from google.protobuf import descriptor as _descriptor
from google.protobuf import message as _message
from google.protobuf import reflection as _reflection
from google.protobuf import symbol_database as _symbol_database
from google.protobuf import descriptor_pb2
# @@protoc_insertion_point(imports)

_sym_db = _symbol_database.Default()




DESCRIPTOR = _descriptor.FileDescriptor(
  name='tfx/proto/example_gen.proto',
  package='tfx.components.example_gen',
  syntax='proto3',
  serialized_pb=_b('\n\x1btfx/proto/example_gen.proto\x12\x1atfx.components.example_gen\"Y\n\x06Output\x12=\n\x0csplit_config\x18\x03 \x01(\x0b\x32\'.tfx.components.example_gen.SplitConfigJ\x04\x08\x01\x10\x02J\x04\x08\x02\x10\x03J\x04\x08\x04\x10\x05\"\x85\x01\n\x0bSplitConfig\x12=\n\x06splits\x18\x01 \x03(\x0b\x32-.tfx.components.exa